# T1569 System Services

-----------------------------------------------------------------------

## Technique Description

Adversaries may abuse system services or daemons to execute commands or programs. Adversaries can execute malicious content by interacting with or creating services either locally or remotely. Many services are set to run at boot, which can aid in achieving persistence ([Create or Modify System Process](https://attack.mitre.org/techniques/T1543)), but adversaries can also abuse services for one-time or temporary execution.

## Technique Detection

Monitor for command line invocations of tools capable of modifying services that doesn’t correspond to normal usage patterns and known software, patch cycles, etc. Also monitor for changes to executables and other files associated with services. Changes to Windows services may also be reflected in the Registry.

-----------------------------------------------------------------------

### Tactics:

  *   Execution

### Platforms:

  * Windows

  * macOS

  * Linux

### Adversary Required Permissions:

  * User

  * Administrator

  * SYSTEM

  * root

### Data Sources:

  * **Windows Registry:** Windows Registry Key Modification

  * **Command:** Command Execution

  * **Service:** Service Creation

  * **File:** File Modification

  * **Process:** Process Creation

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1569)

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 23 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres, SSgt Zachary Burke

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Adversaries will use services to execute malicious commands or payloads.

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| APT28 | | |
| APT29 | | |
| Turla | | |

#### Koadic 	
- Koadic (a Windows post-exploitation framework that has been used by APT28) can run a command on another machine using PsExec.

#### Empire 	
- Empire (open-source post-exploitation framework used by Turla) can use PsExec to execute a payload on a remote host.

#### Cobalt Strike 	
- Cobalt Strike (Has been used by APT29) can use PsExec to execute a payload on a remote host. It can also use Service Control Manager to start new services.

## Detection Blindspots

- There is no SYSMON id for service installations, although one could monitor event id 13’s and where the TargetObject contains \Services\. Additionally, verify network auditing process for Windows Event IDs.

## Analytical References

- https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1569.002/T1569.002.md
- https://docs.microsoft.com/en-us/sysinternals/downloads/psexec
- https://docs.microsoft.com/en-us/windows/win32/services/service-control-manager
- https://github.com/EmpireProject/Empire
- https://github.com/zerosum0x0/koadic

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Changes to service Registry entries and command line invocation of tools capable of modifying services that do not correlate with known software, patch cycles, etc., may be suspicious.
- If a service is used only to execute a binary or script and not to persist, then it will likely be changed back to its original form shortly after the service is restarted so the service is not left broken, as is the case with the common administrator tool PsExec.

#### Analytic 1

  * **Information:** Inspecting service installs (4697/7045) fail-to-starts (7000), and errors (7009).

  * **Source:** SYSMON, Windows Audits

  * **Tool:** Kibana

  * **Notes:** Generally, services should not be newly installed. Look at the “Service File Name: or “ImagePath”.

  * **Query:** ```event.code : (4697 OR 7045 OR 7000 OR 7009)```

#### Analytic 2

  * **Information:** Check command line parameters

  * **Source:** SYSMON, Windows Audits

  * **Tool:** Kibana

  * **Notes:** Add process.command_line and process.parent.command_line to your columns and perform LFA / look for outliars. Feel free to filter on a single binary such as services.exe.

  * **Query:** ```process.name : (net.exe OR net.exe OR psexec.exe OR sc.exe OR services.exe) AND process.command_line: *```


#### Analytic 3

  * **Information:** Look for the services registry key being modified. A new service should appear in this key.

  * **Source:** SYSMON, Windows Audits

  * **Tool:** Kibana

  * **Notes:** Add registry.value and registry.path to your columns.

  * **Query:** ```winlog.event_data.TargetObject: "HKLM\System\CurrentControlSet\Services\*"```

-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

